# 📘 RAG 파이프라인 단계별 실습 (Step-by-Step RAG Pipeline)

이 노트북은 **RAG(Search-Augmented Generation)** 시스템의 전체 과정을 코드 셀 단위로 실행하며 확인하는 튜토리얼입니다.
단순한 결과 확인이 아니라, **데이터 로드 → 텍스트 분할 → 벡터 저장 → 검색 → 리랭킹 → 답변 생성**에 이르는 모든 과정을 상세하게 분해하여 보여줍니다.

### 🚀 학습 목표
1. **데이터 파이프라인**: 실제 RFP 문서들이 어떻게 처리되는지 확인합니다.
2. **다중 문서 검색**: 여러 문서를 동시에 인덱싱하고, 질문에 맞는 문서를 정확히 찾아오는지 테스트합니다.
3. **생성(Generation)**: 다양한 유형의 질문(요약, 비교, 특정 정보 추출)에 대해 AI가 어떻게 답변하는지 확인합니다.

## 1️⃣ 환경 설정 및 라이브러리 로드 (Environment Setup)
프로젝트의 루트 경로를 시스템 경로에 추가하고, `.env` 파일에서 API 키를 로드합니다.

In [1]:
import sys
import os
import glob
from dotenv import load_dotenv

# 프로젝트 루트 경로를 sys.path에 추가 (모듈 임포트를 위해)
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir) if current_dir.endswith('notebooks') else current_dir
sys.path.append(project_root)

# 환경 변수 로드 (OPENAI_API_KEY 확인)
load_dotenv()

print(f"Project Root: {project_root}")
print("환경 변수 로드 완료 ✅")

Project Root: /home/soobeom/codeit_ai_middle_project
환경 변수 로드 완료 ✅


## 2️⃣ 문서 로드 (Data Loading)
다양한 기관의 RFP 문서를 로드하여 테스트 데이터셋을 구성합니다.
**국민연금공단**, **기초과학연구원**, **한국원자력연구원** 등 서로 다른 도메인의 문서를 선정합니다.

In [2]:
from src.ingest.loader import get_loader

# 테스트할 대상 파일 목록 (파일명 일부)
target_keywords = [
    "국민연금공단_2024년 이러닝시스템 운영 용역",
    "기초과학연구원_2025년도 중이온가속기용 극저온시스템",
    "한국원자력연구원_한국원자력연구원 선량평가시스템 고도화"
]

data_dir = os.path.join(project_root, "data", "files")
selected_files = []

# 키워드로 파일 찾기
for keyword in target_keywords:
    # hwp 또는 pdf 확장자 매칭
    matched = glob.glob(os.path.join(data_dir, f"*{keyword}*.*"))
    if matched:
        selected_files.extend(matched)
    else:
        print(f"⚠️ 파일을 찾을 수 없음: {keyword}")

documents = []
print(f"\n📁 총 {len(selected_files)}개의 문서를 로드합니다:")
for file_path in selected_files:
    print(f" - {os.path.basename(file_path)}")
    loader = get_loader(file_path)
    docs = loader.load()
    documents.extend(docs)

print(f"\n✅ 전체 문서 로드 완료 (총 {len(documents)}개 객체)")


📁 총 3개의 문서를 로드합니다:
 - 국민연금공단_2024년 이러닝시스템 운영 용역.hwp
 - 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
 - 한국원자력연구원_한국원자력연구원 선량평가시스템 고도화.hwp

✅ 전체 문서 로드 완료 (총 3개 객체)


## 3️⃣ 텍스트 분할 (Chunking)
로드된 문서들을 LLM이 처리하기 좋은 크기로 분할합니다.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

if documents:
    chunks = splitter.split_documents(documents)
    print(f"🧩 텍스트 분할 결과: 총 {len(chunks)}개의 청크 생성됨")
    
    # 청크 예시 확인
    print("\n--- [Chunk Example] ---")
    print(f"Source: {os.path.basename(chunks[0].metadata['source'])}")
    print(chunks[0].page_content[:200] + "...")
else:
    chunks = []
    print("분할할 문서가 없습니다.")

🧩 텍스트 분할 결과: 총 118개의 청크 생성됨

--- [Chunk Example] ---
Source: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp
<그림>
                                        2024년 이러닝시스템 운영 용역 제안요청서




<표>

 

<표>




2024. 1.




<그림>



<그림>


<그림>


<표>



<표>


1. 사업개요

 □ 사 업 명:『2024년 이러닝시스템 운영』용역

 □ 사업기간: 계약체결일로부터 2025....


## 4️⃣ 벡터 저장소 구축 (Vector Store Indexing)
분할된 청크들을 임베딩하여 ChromaDB에 저장합니다.

In [4]:
from src.common.config import config
from src.indexing.vector_store import VectorStoreWrapper

print("🗄️ 벡터 저장소(Chroma)에 연결 및 데이터 적재...")
vector_wrapper = VectorStoreWrapper(config)
vector_wrapper.initialize()

# 인덱싱 수행
if chunks:
    vector_wrapper.add_documents(chunks)
    print("데이터 적재 완료! ✅")
else:
    print("적재할 데이터가 없습니다.")

경고: 설정을 로드할 수 없습니다: 설정 파일을 찾을 수 없습니다: ./config/local.yaml
🗄️ 벡터 저장소(Chroma)에 연결 및 데이터 적재...
chroma 벡터 스토어를 ./data/index 경로에서 초기화합니다...
벡터 스토어에 118개의 청크를 추가합니다...
문서가 성공적으로 추가되었습니다.
데이터 적재 완료! ✅


## 5️⃣ 질의응답 시뮬레이션 (Q&A Loop)
요청주신 **실제 예시 질문**들을 바탕으로 검색(Retrieval)부터 답변 생성(Generation)까지의 과정을 반복 실행합니다.

### 테스트 질문 목록
1. **국민연금공단**: 이러닝시스템 콘텐츠 개발 관리 요구사항 상세
2. **기초과학연구원**: 극저온시스템 사업 내 AI 기반 예측 요구사항 여부
3. **한국원자력연구원**: 선량 평가 시스템 고도화 추진 목적

In [5]:
from src.retrieval.hybrid import get_hybrid_retriever
from src.retrieval.reranker import FlashrankRerank
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# 1. 검색 엔진 및 모델 초기화
k = 5
hybrid_retriever = get_hybrid_retriever(vector_store=vector_wrapper.vector_store, k=k)
reranker = FlashrankRerank()
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)

# 2. 테스트 질문 정의
test_queries = [
    "국민연금공단이 발주한 이러닝시스템 관련 사업에서 콘텐츠 개발 관리 요구 사항에 대해서 자세히 알려 줘.",
    "기초과학연구원 극저온시스템 사업 요구에서 AI 기반 예측에 대한 요구사항이 있는지, 그리고 모니터링 업무 요청사항은 무엇인지 알려 줘.",
    "한국 원자력 연구원에서 선량 평가 시스템 고도화 사업을 발주했는데, 이 사업이 왜 추진되는지 목적을 알려 줘."
]

# 3. 질문별 실행 루프
for idx, query in enumerate(test_queries, 1):
    print(f"\n{'='*80}")
    print(f"❓ 질문 {idx}: {query}")
    print(f"{'='*80}")
    
    # A. 검색 (Hybrid)
    retrieved_docs = hybrid_retriever.invoke(query)
    print(f"   🔎 [1차 검색] {len(retrieved_docs)}개 문서 검색됨")
    
    # B. 리랭킹 (FlashRank)
    reranked_docs = reranker.compress_documents(documents=retrieved_docs, query=query)
    top_3_docs = reranked_docs[:3]
    
    # 주요 참고 문서 출력
    print(f"   ⚖️ [리랭킹 완료] 상위 {len(top_3_docs)}개 참고 문서:")
    for i, doc in enumerate(top_3_docs):
        score = doc.metadata.get('relevance_score', 0.0)
        source_file = os.path.basename(doc.metadata.get('source', 'Unknown'))
        print(f"      Rank {i+1} (Score: {score:.4f}): {source_file}")

    # C. 답변 생성 (Generation)
    context_text = "\n\n".join([f"[문서 {i+1}] {d.page_content}" for i, d in enumerate(top_3_docs)])
    
    system_prompt = """당신은 전문적인 RFP 분석가입니다. 
    주어진 문맥(Context)에 근거하여 사용자의 질문에 상세하고 정확하게 답변하세요.
    답변 형식을 갖추어 보기 좋게 출력하세요."""

    user_prompt = f"Context:\n{context_text}\n\nQuestion:\n{query}"
    
    print("\n   🤖 [AI 답변 생성 중...]")
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt)
    ])
    
    print(f"\n💬 답변:\n{'-'*60}")
    print(response.content)
    print(f"{'-'*60}\n")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"



❓ 질문 1: 국민연금공단이 발주한 이러닝시스템 관련 사업에서 콘텐츠 개발 관리 요구 사항에 대해서 자세히 알려 줘.
   🔎 [1차 검색] 6개 문서 검색됨
   ⚖️ [리랭킹 완료] 상위 3개 참고 문서:
      Rank 1 (Score: 1.0000): 국민연금공단_2024년 이러닝시스템 운영 용역.hwp
      Rank 2 (Score: 1.0000): 국민연금공단_2024년 이러닝시스템 운영 용역.hwp
      Rank 3 (Score: 1.0000): 국민연금공단_2024년 이러닝시스템 운영 용역.hwp

   🤖 [AI 답변 생성 중...]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 답변:
------------------------------------------------------------
요청하신 대로 국민연금공단(이하 “공단”)이 발주한 이러닝시스템 용역의 “콘텐츠 개발·관리” 요구사항을 문서 내 근거에 따라 정리·요약합니다.

요약
- 핵심 요구는 직무·부서별 교육콘텐츠의 신규개발 및 기존콘텐츠의 적기 업데이트, 영상화(동영상 촬영·실시간 송출 포함) 및 e-book 제작, 자막·스크립트 기능 확대, 품질 유지·보수와 관련한 운영·가격 규정 등입니다.
- 개발 규모·기준(차시·분량)과 유지보수 항목(영상촬영 회수 포함), 외부콘텐츠·교재비 처리 방식, 미이행 시 대금감액 규정 등이 명시되어 있습니다.

세부 요구사항 (문서 근거 기반)

1) 개발 범위 및 수량
- 직무콘텐츠 신규개발: 60차시(동영상 형태 포함)  
- 부서별 필수 콘텐츠: 10차시 개발(동영상 포함)  
- 오프라인 현장(직무전문교수) 강좌의 온라인 전환: 동영상 촬영 및 실시간(화상) 송출 등으로 온라인화

2) 콘텐츠 형식 및 산출물
- 1차시 기준: 30~40분 내외(동영상 타입 포함) — 가격산출 및 단가 기준으로 사용  
- 산출물 예시: 동영상 파일(편집본), 스크립트(자막) 파일, e-book(직무교재·법령·업무매뉴얼 등), 메타데이터/학습모듈 형태(LMS 연동 가능 형식)

3) 자막·스크립트 및 접근성
- 기존 기 개발콘텐츠 및 신규콘텐츠에 스크립트(자막) 기능 확대 필수 → 자막 포함 제공(프레임별 자막 파일 및 내장 자막 제공 등 고려)
- 학습자 접근성 향상(학습자 중심 스마트 학습환경) 지원 요구(화면설계·모바일/태블릿 호환성 등 포함 권장)

4) 촬영·제작·운영 유지보수
- 콘텐츠 및 P/G 유지보수 가격제시에 동영상 촬영 10회(스튜디오 또는 출장 촬영)를 포함하여 제시할 것(제안서상 가격에 포함)  
- 직무교재·법령·업무매뉴얼 등에 대한 동영상 촬영 및 e-book 제작 지원 포함  
- 콘텐츠의 적기 업데이트로 지속적

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


...


💬 답변:
------------------------------------------------------------
요약 결론
- AI 기반 예측(예: 예측진단, 예측유지보수)을 명시한 요구사항은 제공된 문서(문서 1–3) 내에 명확히 존재하지 않습니다. 문서들에서는 “기록 및 분석”과 “모니터링”을 요구하고 있으나, 그 분석 방법에 AI 사용을 필수화하거나 요구하는 문구는 없습니다.

근거 (문서별)
- 문서 1
  - 가동기간 항목: “극저온시스템 모니터링”, “극저온시스템 주요 수치 기록 및 분석”, “극저온시스템 제어” 등 기록·분석·제어를 요구(24시간 운전). (문서 1)
  - 휴지기간 항목: 유지보수 목적의 모니터링·기록·분석 및 운전 매뉴얼 작성·교육 등 포함. (문서 1)
- 문서 2
  - “현장 상황 일시적 또는 지속적 관찰(유지보수 활동 내용 파악 및 진행 상황 파악 포함)” 및 “운전 상황 확인에 필요한 모니터링 — 주기적인 현장 순찰 포함” 등 모니터링 업무 범위를 명시. (문서 2, 표)
  - 비방사선구역 한정으로 연구원 요청 시 관찰 수행. (문서 2)
- 문서 3
  - 가동·비상 상황 관련 책임 및 통보 절차(긴급상황 시 즉시 연구원에게 전파 및 지시 이행 등)를 규정. (문서 3)
  - 휴지기간에는 1~2인이 유지보수 등을 위한 극저온시스템 모니터링 및 제어를 수행하도록 명시(나머지 인력은 경험 취합 등). (문서 3)

모니터링 업무 요청사항(세부)
- 가동기간(기본: 24시간)
  - 지속적(24시간) 시스템 모니터링
  - 주요 계측값(주요 수치) 기록 및 분석
  - 시스템 제어(운전·조작)
  - 기타 가동기간에 필요한 운전 업무(비상발생 시 연구원에 즉시 통보 및 연구원 지시 이행 의무 포함)
- 휴지기간(주간 운전 기준)
  - 유지보수 지원을 위한 모니터링 및 제어(주간)
  - 주요 수치 기록 및 분석
  - 운전 매뉴얼 작성·개정
  - 운전 교육·성과보고회 등
  - 주기적 현장 순찰 및 필요시 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


   🔎 [1차 검색] 8개 문서 검색됨
   ⚖️ [리랭킹 완료] 상위 3개 참고 문서:
      Rank 1 (Score: 1.0000): 한국수자원공사_용인 첨단 시스템반도체 국가산단 용수공급사업 타당성.hwp
      Rank 2 (Score: 1.0000): 한국원자력연구원_한국원자력연구원 선량평가시스템 고도화.hwp
      Rank 3 (Score: 1.0000): 한국수자원공사_용인 첨단 시스템반도체 국가산단 용수공급사업 타당성.hwp

   🤖 [AI 답변 생성 중...]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 답변:
------------------------------------------------------------
요약
- 본 사업은 규제요건 준수와 선량평가 기법 개선, 액체유출물에 의한 주민피폭 평가 기능 추가, 업무생산성 향상을 위해 추진됩니다.

세부 목적
1. 규제요건 준수
   - 원자력안전위원회고시 제2019-10호(「방사선방호 등에 관한 기준」) 제16조에 따라 제한구역 경계에서의 연간선량 준수 여부를 확인할 수 있는 체계 구축이 필요합니다. (문서 2, 1.2 가)

2. 선량평가 기법·모델 개선
   - 기존 ICRP60 기반의 평가장기(Organ) 모델을 개선하여 보다 적절하고 최신의 평가체계를 마련하려는 목적입니다. (문서 2, 1.2 가, 1.3 가)

3. 액체유출물에 의한 선량평가 기능 추가
   - 액체유출물로 인한 주민피폭 선량평가를 수행할 수 있는 기능을 포트란 모듈 및 웹시스템에 추가하여 평가범위를 확대하려는 목적입니다. (문서 2, 1.2 가·나, 1.3 나)

4. 업무생산성 및 의사결정 지원
   - 사용자 인터페이스(UI) 개선과 출력 기능 고도화를 통해 업무 효율을 높이고 신속한 의사결정 환경을 제공하려는 목적입니다. (문서 2, 1.2 나)

기대효과 (연계)
- 정상운전 시 규제 수요 대응 가능, 선량평가 시스템 고도화를 통한 데이터 신뢰성 확보 및 업무 생산성 향상. (문서 2, 1.4)
------------------------------------------------------------

